In [1]:
import carla
import numpy as np
from car_dreamer.toolkit.planner.agents.navigation.global_route_planner import GlobalRoutePlanner
from car_dreamer.toolkit.planner.agents.navigation.basic_agent import BasicAgent
from car_dreamer.toolkit.carla_manager import WorldManager

from navigator.wp_utils import xyz_to_right_lane
from navigator.visualize import CarlaVisualizer
from navigator.tsp_solver import optimize_route_order
from navigator.route_generator import generate_full_route
from navigator.test import spawn_traffic

from car_dreamer.toolkit import Config
from car_dreamer import create_task
from dreamerv3.agent import Agent as DreamerV3Agent
from car_dreamer.toolkit.observer.observer import Observer  
from car_dreamer import load_task_configs
from dreamerv3 import embodied
from dreamerv3.embodied.core import Driver

import os  
# import torch  
from car_dreamer.toolkit.observer.observer import Observer  
from car_dreamer.toolkit.observer.handlers import BirdeyeHandler  
from dreamerv3 import embodied  
from copy import deepcopy

/home/zeyadcode_jammy/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
2025-05-08 11:43:56.999243: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
client = carla.Client("localhost", 2000)
client.set_timeout(20)
world = client.get_world()
carla_map = world.get_map()
ego_vehicle = next(v for v in world.get_actors().filter("vehicle.*") if v.attributes.get("role_name") == "ego_vehicle")

spawn_points = carla_map.get_spawn_points()
target_points = [
    [280.363739, 133.306351, 0.001746],
    [334.949799, 161.106171, 0.001736],
    [339.100037, 258.568939, 0.001679],
    [396.295319, 183.195740, 0.001678],
    [267.657074, 1.983160, 0.001678],
    [153.868896, 26.115866, 0.001678],
    [290.515564, 56.175072, 0.001677],
    [92.325722, 86.063644, 0.001677],
    [88.384346, 287.468567, 0.001728],
    [177.594101, 326.386902, 0.001677],
    [-1.646942, 197.501282, 0.001555],
    [59.701321, 1.970804, 0.001467],
    [122.100121, 55.142044, 0.001596],
    [161.030975, 129.313187, 0.001679],
    [184.758713, 199.424271, 0.001680],
]
sampling_resolution = 1.0

target_locations = xyz_to_right_lane(target_points, carla_map)

grp = GlobalRoutePlanner(carla_map, sampling_resolution)

optimized_targets = optimize_route_order(
    ego_vehicle.get_location(),
    target_locations,
    grp,
)

full_route = generate_full_route(ego_vehicle.get_location(), optimized_targets, grp)

Calculating pairwise distances...


In [8]:
cfg = load_task_configs('carla_navigation')
env_cfg = cfg.env
observer_cfg = env_cfg.observation

In [11]:
world_manager = WorldManager(env_cfg, client=client)

[CARLA] Connecting to Carla server at 2000...
[CARLA] Map Town04 loaded


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 world_manager = WorldManager(env_cfg, client=client)                                         │
│   2                                                                                              │
│                                                                                                  │
│ /home/zeyadcode_jammy/.local/lib/python3.10/site-packages/car_dreamer/toolkit/carla_manager/worl │
│ d_manager.py:54 in __init__                                                                      │
│                                                                                                  │
│    51 │   │   self._settings = settings                                                          │
│    52 │   │                                                                                      │
│    53 │   │   self._tm_port = self._config.carla_port + 6000                                     │
│ ❱  54 │   │   self._vehicle_manager = VehicleManager(self._client, self._tm_port, self._config   │
│    55 │   │                                                                                      │
│    56 │   │   self._on_reset = None                                                              │
│    57 │   │   self._apply_control = None                                                         │
│                                                                                                  │
│ /home/zeyadcode_jammy/.local/lib/python3.10/site-packages/car_dreamer/toolkit/carla_manager/vehi │
│ cle_manager.py:7 in __init__                                                                     │
│                                                                                                  │
│    4 │   │   self._traffic_config = traffic_config                                               │
│    5 │   │                                                                                       │
│    6 │   │   self._tm = self._client.get_trafficmanager(tm_port)                                 │
│ ❱  7 │   │   self._tm.set_global_distance_to_leading_vehicle(3.5)                                │
│    8 │   │   self._tm.set_respawn_dormant_vehicles(True)                                         │
│    9 │   │   self._tm.set_boundaries_respawn_dormant_vehicles(40, 100)                           │
│   10 │   │   self._tm.set_random_device_seed(traffic_config.tm_seed)                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: rpc::timeout: Timeout of 2000ms while calling RPC function 'set_global_distance_to_leading_vehicle'

In [12]:
observer = Observer(world_manager, observer_cfg)

In [14]:
# Load the pretrained DreamerV3 model  
checkpoint_path = "./navigation.ckpt"  # Path to your downloaded checkpoint  
  
# Create DreamerV3 config  
config = embodied.Config()  
config = config.update({  
    "logdir": "./logdir/eval",  
    "jax": {  
        "policy_devices": [0],  
        "train_devices": [0]  
    },  
    "encoder": {  
        "cnn_keys": "birdeye_wpt"  
    },  
    "decoder": {  
        "cnn_keys": "birdeye_wpt"  
    }  
})  

In [15]:
# Load the pretrained DreamerV3 model  
checkpoint_path = "./navigation.ckpt"  # Path to your downloaded checkpoint  
  
# Create DreamerV3 config  
config = embodied.Config()  
config = config.update({  
    "logdir": "./logdir/eval",  
    "jax": {  
        "policy_devices": [0],  
        "train_devices": [0]  
    },  
    "encoder": {  
        "cnn_keys": "birdeye_wpt"  
    },  
    "decoder": {  
        "cnn_keys": "birdeye_wpt"  
    }  
})  
  
# Create DreamerV3 agent  
agent = embodied.make_agent(config)  
agent.load(checkpoint_path)  

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:21                                                                                   │
│                                                                                                  │
│   18 })                                                                                          │
│   19                                                                                             │
│   20 # Create DreamerV3 agent                                                                    │
│ ❱ 21 agent = embodied.make_agent(config)                                                         │
│   22 agent.load(checkpoint_path)                                                                 │
│   23                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: module 'dreamerv3.embodied' has no attribute 'make_agent'